In [ ]:
# Change notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

# Problem 1

In [1]:
from dataclasses import dataclass
from typing import Tuple, Dict, List, Optional, Iterable, Mapping, Sequence
from rl.markov_decision_process import *
from rl.markov_process import *
from rl.distribution import *
from rl.function_approx import DNNSpec, AdamGradient, DNNApprox
from rl.approximate_dynamic_programming import back_opt_vf_and_policy
from rl.approximate_dynamic_programming import back_opt_qvf
from operator import itemgetter
from pprint import pprint
import numpy as np
import itertools
import time

In [7]:
class RiskyDistribution(SampledDistribution[Tuple[float, float]]):
    """
    A class container for the risky returns distribution.
    Super's into the SampledDistribution class with a function sampler and expectation values.
    Since we know the Risky Return Distribution we can calculate it directly instead
    of taking a bunch of sequential samples
    """
    def __init__(self, mu, sigma, reward):
        # Get mu, sigma, and reward
        self.mu:     float = mu
        self.sigma:  float = sigma
        self.reward: float = reward
            
        # Create a sampler of a tuple of a random normal value with 
        # mean mu and variance sigma coupled with a reward
        dis_sampler = lambda : (np.random.normal(mu, sigma), reward)
        
        # Feed the sampler and one single sample to take to the SampledDistribution class
        super().__init__(sampler=dis_sampler,  expectation_samples=1)
    
        

In [8]:
@dataclass(frozen=True)
class AssetAllocDiscrete:
    risky_return_distributions: Sequence[Distribution[float]]
    riskless_returns: Sequence[float]
    utility_func: Callable[[float], float]
    risky_alloc_choices: Sequence[float]
    feature_functions: Sequence[Callable[[Tuple[float, float]], float]]
    dnn_spec: DNNSpec
    initial_wealth_distribution: Distribution[float]

    def time_steps(self) -> int:
        return len(self.risky_return_distributions)

    def uniform_actions(self) -> Choose[float]:
        return Choose(set(self.risky_alloc_choices))

    def get_mdp(self, t: int) -> MarkovDecisionProcess[float, float]:
        """
        State is Wealth W_t, Action is investment in risky asset (= x_t)
        Investment in riskless asset is W_t - x_t
        """

        distr: Distribution[float] = self.risky_return_distributions[t]
        rate: float = self.riskless_returns[t]
        alloc_choices: Sequence[float] = self.risky_alloc_choices
        steps: int = self.time_steps()
        utility_f: Callable[[float], float] = self.utility_func

        class AssetAllocMDP(MarkovDecisionProcess[float, float]):

            def step(
                self,
                wealth: float,
                alloc: float
# Changed Range vv
            ) -> RiskyDistribution:
                
                # Calculate the expected wealth
                wealth_expectation = (alloc*distr.μ) + (wealth-alloc)*(1+rate) 
                # Calculate the corresponding reward at time step t
                reward: float = utility_f(wealth_expectation) if t == steps - 1 else 0.  
                    
                wealth_distribution = RiskyDistribution(wealth_expectation, distr.σ, reward)
                    
                return wealth_distribution
# Changed Range ^^

            def actions(self, wealth: float) -> Sequence[float]:
                return alloc_choices

        return AssetAllocMDP()

    def get_qvf_func_approx(self) -> DNNApprox[Tuple[float, float]]:

        adam_gradient: AdamGradient = AdamGradient(
            learning_rate=0.1,
            decay1=0.9,
            decay2=0.999
        )
        return DNNApprox.create(
            feature_functions=self.feature_functions,
            dnn_spec=self.dnn_spec,
            adam_gradient=adam_gradient
        )

    def get_states_distribution(self, t: int) -> SampledDistribution[float]:

        actions_distr: Choose[float] = self.uniform_actions()

        def states_sampler_func() -> float:
            wealth: float = self.initial_wealth_distribution.sample()
            for i in range(t):
                distr: Distribution[float] = self.risky_return_distributions[i]
                rate: float = self.riskless_returns[i]
                alloc: float = actions_distr.sample()
                wealth = alloc * (1 + distr.sample()) + \
                    (wealth - alloc) * (1 + rate)
            return wealth

        return SampledDistribution(states_sampler_func)

    def backward_induction_qvf(self) -> \
            Iterator[DNNApprox[Tuple[float, float]]]:

        init_fa: DNNApprox[Tuple[float, float]] = self.get_qvf_func_approx()

        mdp_f0_mu_triples: Sequence[Tuple[
            MarkovDecisionProcess[float, float],
            DNNApprox[Tuple[float, float]],
            SampledDistribution[float]
        ]] = [(
            self.get_mdp(i),
            init_fa,
            self.get_states_distribution(i)
        ) for i in range(self.time_steps())]

        num_state_samples: int = 300
        error_tolerance: float = 1e-6

        return back_opt_qvf(
            mdp_f0_mu_triples=mdp_f0_mu_triples,
            γ=1.0,
            num_state_samples=num_state_samples,
            error_tolerance=error_tolerance
        )

    def get_vf_func_approx(
        self,
        ff: Sequence[Callable[[float], float]]
    ) -> DNNApprox[float]:

        adam_gradient: AdamGradient = AdamGradient(
            learning_rate=0.1,
            decay1=0.9,
            decay2=0.999
        )
        return DNNApprox.create(
            feature_functions=ff,
            dnn_spec=self.dnn_spec,
            adam_gradient=adam_gradient
        )

    def backward_induction_vf_and_pi(
        self,
        ff: Sequence[Callable[[float], float]]
    ) -> Iterator[Tuple[DNNApprox[float], Policy[float, float]]]:

        init_fa: DNNApprox[float] = self.get_vf_func_approx(ff)

        mdp_f0_mu_triples: Sequence[Tuple[
            MarkovDecisionProcess[float, float],
            DNNApprox[float],
            SampledDistribution[float]
        ]] = [(
            self.get_mdp(i),
            init_fa,
            self.get_states_distribution(i)
        ) for i in range(self.time_steps())]

        num_state_samples: int = 300
        error_tolerance: float = 1e-8

        return back_opt_vf_and_policy(
            mdp_f0_mu_triples=mdp_f0_mu_triples,
            γ=1.0,
            num_state_samples=num_state_samples,
            error_tolerance=error_tolerance
        )

In [11]:
#Main
steps: int = 4
μ: float = 0.13
σ: float = 0.2
r: float = 0.07
a: float = 1.0
init_wealth: float = 1.0
init_wealth_var: float = 0.1

excess: float = μ - r
var: float = σ * σ
base_alloc: float = excess / (a * var)

risky_ret: Sequence[Gaussian] = [Gaussian(μ=μ, σ=σ) for _ in range(steps)]
riskless_ret: Sequence[float] = [r for _ in range(steps)]
utility_function: Callable[[float], float] = lambda x: - np.exp(-a * x) / a
alloc_choices: Sequence[float] = np.linspace(
    2 / 3 * base_alloc,
    4 / 3 * base_alloc,
    11
)
feature_funcs: Sequence[Callable[[Tuple[float, float]], float]] = \
    [
        lambda _: 1.,
        lambda w_x: w_x[0],
        lambda w_x: w_x[1],
        lambda w_x: w_x[1] * w_x[1]
    ]
dnn: DNNSpec = DNNSpec(
    neurons=[],
    bias=False,
    hidden_activation=lambda x: x,
    hidden_activation_deriv=lambda y: np.ones_like(y),
    output_activation=lambda x: - np.sign(a) * np.exp(-x),
    output_activation_deriv=lambda y: -y
)
init_wealth_distr: Gaussian = Gaussian(μ=init_wealth, σ=init_wealth_var)

aad: AssetAllocDiscrete = AssetAllocDiscrete(
    risky_return_distributions=risky_ret,
    riskless_returns=riskless_ret,
    utility_func=utility_function,
    risky_alloc_choices=alloc_choices,
    feature_functions=feature_funcs,
    dnn_spec=dnn,
    initial_wealth_distribution=init_wealth_distr
)

# vf_ff: Sequence[Callable[[float], float]] = [lambda _: 1., lambda w: w]
# it_vf: Iterator[Tuple[DNNApprox[float], Policy[float, float]]] = \
#     aad.backward_induction_vf_and_pi(vf_ff)

# print("Backward Induction: VF And Policy")
# print("---------------------------------")
# print()
# for t, (v, p) in enumerate(it_vf):
#     print(f"Time {t:d}")
#     print()
#     opt_alloc: float = p.act(init_wealth).value
#     val: float = v.evaluate([init_wealth])[0]
#     print(f"Opt Risky Allocation = {opt_alloc:.2f}, Opt Val = {val:.3f}")
#     print("Weights")
#     for w in v.weights:
#         print(w.weights)
#     print()

it_qvf: Iterator[DNNApprox[Tuple[float, float]]] = aad.backward_induction_qvf()
    

print("Backward Induction on Q-Value Function")
print("--------------------------------------")
print()
for t, q in enumerate(it_qvf):
    print(f"Time {t:d}")
    print()
    opt_alloc: float = max(
        ((q.evaluate([(init_wealth, ac)])[0], ac) for ac in alloc_choices),
        key=itemgetter(0)
    )[1]
    val: float = max(q.evaluate([(init_wealth, ac)])[0]
                     for ac in alloc_choices)
    print(f"Opt Risky Allocation = {opt_alloc:.3f}, Opt Val = {val:.3f}")
    print("Optimal Weights below:")
    for wts in q.weights:
        pprint(wts.weights)
    print()

print("Analytical Solution")
print("-------------------")
print()

for t in range(steps):
    print(f"Time {t:d}")
    print()
    left: int = steps - t
    growth: float = (1 + r) ** (left - 1)
    alloc: float = base_alloc / growth
    val: float = - np.exp(- excess * excess * left / (2 * var)
                          - a * growth * (1 + r) * init_wealth) / a
    bias_wt: float = excess * excess * (left - 1) / (2 * var) + \
        np.log(np.abs(a))
    w_t_wt: float = a * growth * (1 + r)
    x_t_wt: float = a * excess * growth
    x_t2_wt: float = - var * (a * growth) ** 2 / 2

    print(f"Opt Risky Allocation = {alloc:.3f}, Opt Val = {val:.3f}")
    print(f"Bias Weight = {bias_wt:.3f}")
    print(f"W_t Weight = {w_t_wt:.3f}")
    print(f"x_t Weight = {x_t_wt:.3f}")
    print(f"x_t^2 Weight = {x_t2_wt:.3f}")
    print()

Backward Induction on Q-Value Function
--------------------------------------

Time 0

Opt Risky Allocation = 1.000, Opt Val = -19.362
Optimal Weights below:
array([[-3.07409814,  1.37970284, -1.31129987,  0.04238005]])

Time 1

Opt Risky Allocation = 1.000, Opt Val = -6.263
Optimal Weights below:
array([[-2.06243504,  1.26386913, -1.01412592, -0.02190888]])

Time 2

Opt Risky Allocation = 1.000, Opt Val = -2.289
Optimal Weights below:
array([[-0.90812639,  1.13310174, -1.08980338,  0.03676592]])

Time 3

Opt Risky Allocation = 1.000, Opt Val = -0.878
Optimal Weights below:
array([[-3.00438357e-04,  1.07000010e+00, -9.39598087e-01,
        -1.29889411e-04]])

Analytical Solution
-------------------

Time 0

Opt Risky Allocation = 1.224, Opt Val = -0.225
Bias Weight = 0.135
W_t Weight = 1.311
x_t Weight = 0.074
x_t^2 Weight = -0.030

Time 1

Opt Risky Allocation = 1.310, Opt Val = -0.257
Bias Weight = 0.090
W_t Weight = 1.225
x_t Weight = 0.069
x_t^2 Weight = -0.026

Time 2

Opt Risky A

# Problem 3

### States
States = (Employment, Skill) where Employment = {Employed, Unemployed} and skill is a real numbered value.  In modelling the MDP, the skill can be either continuous or discrete but practically the skill level should be discrete.  A maximum skill level could also be imposed on the skill level in order to further limit the size of the statespace.  This also makes sense in real life because usually workers gain up to a certain skill level in their careers.  Unless you're a super genius or something there is no infinite skill level.  I will pick $ skill \in [0,100] $ 
<br> <br>
### Actions
Actions = the fraction f spent working vs learning where f is realistically a continuous variable although in solving the MDP, f can be discretized
$f \in [0, 1]$ 
<br> <br>
### Transition Probabilities
For given employment E or U and given skill $s_0$ and higher skill $s_1$ and lower skill $s_{-1}$
State transisions matched to probabilities: <br>
{E, $s_0$} -> {U, $s_1$}     Probability of losing job = p <br>
{E, $s_0$} -> {E, $s_1$}     Probability of not losing job = 1 - p <br>
{U, $s_0$} -> {E, $s_{-1}$}  Probability of being offered a job = h(s) <br>
{U, $s_0$} -> {E, $s_{-1}$}  Probability of not being offered a job = 1 - h(s) 
<br> <br>
Given the skill decay with half life $\lambda$,  $s(t) = s_0 e^{-\lambda t}$ where t is the time unemployed for each day and skill increasing given by g(s)  
$s_1 = g(s_0)$ <br>
$s_{-1} = s_0 e^{-\lambda t} $ 
<br> <br>
### Rewards
If employed R = f(s) * hours working <br>
If unemployed R = 0
<br> <br>

### Variations
For a finite horizon vs infinite horizon, initially the policy should be the same until close to the end of the horizon where I would guess the policy of the finite horizon would choose to work more and learn less to maximize the wages before the end of the simulation.  
<br> <br>
For multiple skills, the state tuple would get bigger making the state space much larger, and making the MDP harder to solve.  The action space would also change to a tuple of the fraction of (employment, skill_1, skill_2, ... skill_n-1) since you can figure out the fraction of skill_n based on the other skills.  The reward model would also change since the wage function f could pay differing amounts for different skills potentially prioritizing certain skills over others. Learning certain skills would also affect the types of jobs offered.
<br> <br>
For multiple jobs you introduce the possibility of accepting or rejecting different jobs with different wage reward models, adding another action object to keep track of.  
<br> <br>
Adding consumption costs could add a fixed negative reward for being both employed and unemployed. There would also be an added penalty for consuming more than the total wages you have.  